In [1]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

import os
import re
import json
import glob
from copy import deepcopy
from collections import defaultdict
from functools import partial
from imblearn.under_sampling import RandomUnderSampler

import pandas as pd
import numpy as np

from nltk import sent_tokenize

import matplotlib.pyplot as plt
import seaborn as sns

import unidecode

from tqdm.notebook import tqdm
import string

from transformers import AutoTokenizer, AutoModel
import torch
from torch import nn
from torchcrf import CRF

from sklearn.model_selection import train_test_split

%matplotlib inline

device='cuda' if torch.cuda.is_available() else 'cpu'
# model_checkpoint='/kaggle/input/huggingface-bert/bert-base-uncased/'
model_checkpoint='bert-large-uncased'
tokenizer=AutoTokenizer.from_pretrained(model_checkpoint)

def clean_text(txt):
    return [re.sub('[^A-Za-z0-9]+', ' ', t.lower()) for t in txt]

torch.manual_seed(1)

In [2]:
# train_df=pd.read_csv('../input/show-me-the-data-preparer/processed_train_df.csv')
# sample_sub = pd.read_csv('../input/show-me-the-data-preparer/processed_test_df.csv')

train_df=pd.read_csv('processed_train_df.csv').reset_index()
sample_sub = pd.read_csv('processed_test_df.csv')

train_df.drop_duplicates(inplace=True)
train_df.dropna(inplace=True)
# train_df.label=train_df.label.fillna('')
# labels=[int(len(label)>0) for label in train_df.label]

# rus=RandomUnderSampler()
# X_res, y_res = rus.fit_resample(train_df.iloc[:,:2], labels)
# train_df=pd.merge(train_df,X_res.drop('text',axis=1),on='index').set_index('index')

train_df=train_df[[len(str(t)) > 50 for t in train_df.text]]

X_train, X_val, y_train, y_val = train_test_split(train_df.text, train_df.label, random_state=1821)

In [3]:
START_TAG=tokenizer.cls_token
STOP_TAG=tokenizer.sep_token
PAD_TAG=tokenizer.pad_token

tag_to_ix = {"B": 0, "I": 1, "O": 2, START_TAG: 3, STOP_TAG: 4, PAD_TAG:-1}

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class my_model(nn.Module):
    def __init__(self,backbone,tag_to_ix):
        super(my_model,self).__init__()
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        # feature extraction
        self.backbone=backbone
        self.hidden_dim=backbone(**tokenizer('test',return_tensors='pt'))[0].shape[-1]
        # Maps the output of the backbone into tag space.
        self.hidden2tag = nn.Linear(self.hidden_dim, self.tagset_size)
        self.aux_fc = nn.Linear(self.hidden_dim,1)
        # CRF
        self.crf = CRF(self.tagset_size, batch_first=True)

    def forward(self, inputs, labels=None):
        # Get the emission scores from the backbone
        outputs = self.backbone(**inputs)[0]
        emission = self.hidden2tag(outputs)
        # Return result
        if labels is not None:
            loss = -self.crf(nn.functional.log_softmax(emission,2), labels, mask=inputs['attention_mask'].bool(), reduction='mean')
            return loss
        else:
            prediction = self.crf.decode(emission,mask=inputs['attention_mask'].bool())
            return prediction

def gen_label(text,label):
    encoded_text=[tokenizer.cls_token] + tokenizer.tokenize(text) + [tokenizer.sep_token]
    result=[tokenizer.cls_token] + ['O']*len(tokenizer.tokenize(text)) + [tokenizer.sep_token]
    for label in label:
        if label=='':
            continue
        encoded_label=tokenizer.tokenize(label)
        for i,token in enumerate(encoded_text):
            if token==encoded_label[0] and encoded_text[i:i+len(encoded_label)]==encoded_label:
                result[i]='B'
                result[i+1:i+len(encoded_label)]=['I']*(len(encoded_label)-1)
    return [tag_to_ix[i] for i in result]

def gen_label_batch(texts,labels):
    tags=[gen_label(*inputs)[:max_len] for inputs in zip(texts,labels)]
    max_length=max([len(tag) for tag in tags])
    if tokenizer.padding_side=='right':
        return torch.tensor([tag+[tag_to_ix[PAD_TAG]]*(max_length-len(tag)) for tag in tags], dtype=torch.long, device=device).view(len(texts),-1)
    else:
        return torch.tensor([[tag_to_ix[PAD_TAG]]*(max_length-len(tag))+tag for tag in tags], dtype=torch.long, device=device).view(len(texts),-1)

In [4]:
class my_ds(torch.utils.data.Dataset):
    def __init__(self,text,label):
        super().__init__()
        self.text=text
        self.label=label
    
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self,idx):
        return self.text.iloc[idx], self.label.iloc[idx]

In [5]:
train_ds=my_ds(X_train,y_train)
val_ds=my_ds(X_val,y_val)

In [6]:
train_loader=torch.utils.data.DataLoader(train_ds,shuffle=True,batch_size=4)
val_loader=torch.utils.data.DataLoader(val_ds,shuffle=False,batch_size=4)

In [7]:
from transformers import get_scheduler, AdamW

max_len=128

def train_fn(train_loader,model,optimizer,lr_scheduler):
    model.train()
    train_loss=0
    train_epoch=tqdm(train_loader, total = len(train_loader), leave=False)

    for index, batch in enumerate(train_epoch):
        # gen input
        text=list(batch[0])
        inputs=tokenizer(text,return_tensors='pt',padding=True,truncation=True,max_length=max_len)
        inputs={k:v.to(device) for k,v in inputs.items()}
        # gen label
        data_names=[label.strip().split('|') for label in batch[1]]
        labels=torch.tensor([len(label)>0 for label in data_names],dtype=torch.float,device=device).view(len(data_names),-1)
        tags=gen_label_batch(text,data_names)
        
        # get loss
        loss = model(inputs, tags)
        
        # optimizing
        loss.backward()
        optimizer.step()   
        lr_scheduler.step()
        optimizer.zero_grad()
             
        train_loss += loss.detach()

        if index % 10 == 0:
            train_epoch.set_description('Step:{} | Loss:{:.3f}'.format(index,loss.item()))
    return train_loss.sum()

def eval_fn(val_loader,model):
    model.eval()
    eval_loss=0
    
    with torch.no_grad():
        for index, batch in enumerate(tqdm(val_loader, total = len(val_loader), leave=False)):
            # gen input
            text=list(batch[0])
            inputs=tokenizer(text,return_tensors='pt',padding=True,truncation=True,max_length=max_len)
            inputs={k:v.to(device) for k,v in inputs.items()}
            # gen label
            data_names=[label.strip().split('|') for label in batch[1]]
            labels=torch.tensor([len(label)>0 for label in data_names],dtype=torch.float,device=device).view(len(data_names),-1)
            tags=gen_label_batch(text,data_names)
            
            # get loss
            loss = model(inputs, tags)
            
            eval_loss += loss.detach()
            
    return eval_loss.sum()

def train_engine(model, epoch, train_loader, val_loader):
    # model = torch.nn.DataParallel(model)
    optimizer = AdamW(model.parameters(), lr=1e-4)
    
    num_training_steps = epoch * len(train_loader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=100,
        num_training_steps=num_training_steps
    )
    
    best_eval_loss = np.inf
    for i in tqdm(range(epoch)):
        train_loss = train_fn(train_loader,model,optimizer,lr_scheduler)
        eval_loss = eval_fn(val_loader, model)
        
        print(f"Epoch {i} , Train loss: {train_loss}, Eval loss: {eval_loss}")

        if eval_loss < best_eval_loss:
            best_eval_loss = eval_loss           
            
            print("Saving the model")
            torch.save(model.state_dict(), model_checkpoint+'.bin')
            
#     return model, eval_predictions, true_labels 


In [8]:
backbone=AutoModel.from_pretrained(model_checkpoint)
model=my_model(backbone,tag_to_ix).to(device)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
train_engine(model,10,train_loader,val_loader)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/9066 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (521 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/3023 [00:00<?, ?it/s]

Epoch 0 , Train loss: 185582.546875, Eval loss: 45582.32421875
Saving the model


...

  0%|          | 0/3023 [00:00<?, ?it/s]

Epoch 8 , Train loss: 63941.83203125, Eval loss: 47015.40625


  0%|          | 0/9066 [00:00<?, ?it/s]

  0%|          | 0/3023 [00:00<?, ?it/s]

Epoch 9 , Train loss: 63817.1328125, Eval loss: 52570.28125
